# بسم الله الرحمن الرحيم

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms

import json
import cv2
import numpy as np
import zipfile
devic = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
with zipfile.ZipFile('/content/drive/MyDrive/Data set/tennis_court_det_dataset.zip' , 'r') as f :
  f.extractall('/content/TennisData')

In [ ]:
class KeypointsDataset(Dataset):
    def __init__(self, img_dir, data_file):
        self.img_dir = img_dir
        with open(data_file, "r") as f:
            self.data = json.load(f)

        self.transforms = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        img = cv2.imread(f"{self.img_dir}/{item['id']}.png")
        h,w = img.shape[:2]

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = self.transforms(img)
        kps = np.array(item['kps']).flatten()
        kps = kps.astype(np.float32)

        kps[::2] *= 224.0 / w # Adjust x coordinates
        kps[1::2] *= 224.0 / h # Adjust y coordinates

        return img, kps

In [ ]:
train_dataset = KeypointsDataset("/content/TennisData/data/images","/content/TennisData/data/data_train.json")
val_dataset = KeypointsDataset("/content/TennisData/data/images","/content/TennisData/data/data_val.json")

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=True)

In [ ]:
model = models.resnet50(pretrained=True)
model.fc =  torch.nn.Linear(model.fc.in_features, 14*2) # Replaces the last layer

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 129MB/s]


In [ ]:
model = model.to(devic)

In [ ]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [11]:
epochs=20
for epoch in range(epochs):
    for i, (imgs,kps) in enumerate(train_loader):
        imgs = imgs.to(devic)
        kps = kps.to(devic)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, kps)
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch}, iter {i}, loss: {loss.item()}")

Epoch 0, iter 828, loss: 204.71182250976562
Epoch 1, iter 828, loss: 18.124347686767578
Epoch 2, iter 828, loss: 16.66643714904785
Epoch 3, iter 828, loss: 9.161162376403809
Epoch 4, iter 828, loss: 52.74058532714844
Epoch 5, iter 828, loss: 16.72909927368164
Epoch 6, iter 828, loss: 2.0652616024017334
Epoch 7, iter 828, loss: 4.2975568771362305
Epoch 8, iter 828, loss: 1.46559476852417
Epoch 9, iter 828, loss: 1.019932746887207
Epoch 10, iter 828, loss: 1.3354132175445557
Epoch 11, iter 828, loss: 0.929623007774353
Epoch 12, iter 828, loss: 1.7763521671295166
Epoch 13, iter 828, loss: 5.046415328979492
Epoch 14, iter 828, loss: 0.8368075489997864
Epoch 15, iter 828, loss: 1.9738987684249878
Epoch 16, iter 828, loss: 2.9031198024749756
Epoch 17, iter 828, loss: 2.8941245079040527
Epoch 18, iter 828, loss: 2.9193115234375
Epoch 19, iter 828, loss: 1.4006022214889526


In [12]:
model.eval()  # Set the model to evaluation mode
total_loss = 0.0
with torch.no_grad():  # Disable gradient calculation
    for imgs, kps in val_loader:
        imgs = imgs.to(devic)
        kps = kps.to(devic)
        outputs = model(imgs)
        loss = criterion(outputs, kps)
        total_loss += loss.item()

avg_loss = total_loss / len(val_loader)
print(f"Validation Loss: {avg_loss}")

Validation Loss: 5.555735425183059


In [13]:
torch.save(model.state_dict(), 'tennis_keypoint_model.pth')
print("Model saved successfully!")

Model saved successfully!
